# 02. Agent Workflow Demo

This notebook demonstrates the complete workflow of AgentCore, from planning to execution.

## Workflow Steps
1. Task Planning
2. Plan Validation
3. Execution
4. Result Processing
5. Memory Storage

In [ ]:
# Cell 1: Add project root to Python path
import sys
import os

# Get the absolute path to the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

# Print the updated path for verification
# print("Updated Python path:")
# for path in sys.path:
#     print(path)
from agents.planning_agent import PlanningAgent
from agents.execution_agent import ExecutionAgent
from memory.ollama_embedding import get_ollama_embedding
from memory.vector_memory import VectorMemory
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

In [ ]:
#ToDo: Store in /configs/planning.yaml and read
planning_config = {
    "model": "gpt-4",
    "temperature": 0.7,
    "max_tokens": 1000,
    "system_prompt": "You are a planning agent that breaks down tasks into executable steps."
}

# Import tools from framework
from tools import SearchTool
import asyncio

# Initialize tools
tools = [SearchTool()]

#ToDo: Store in /configs/execution.yaml and read
execution_config = {
    "model": "gpt-4",
    "temperature": 0.7,
    "max_tokens": 1000,
    "system_prompt": "You are an execution agent that carries out tasks step by step."
}

#ToDo: Store in /configs/memory.yaml and read
memory_config = {
    "embedding_model": "ollama:mxbai-embed-large",
    "index_name": "agent_memory",
    "dimension": 1024 #Hard output limit for mxbai-embed-large
}

# Initialize components with configuration
planning_agent = PlanningAgent(config=planning_config)
execution_agent = ExecutionAgent(config=execution_config, tools=tools)
vector_memory = VectorMemory(config=memory_config, embedding_fn=get_ollama_embedding)

# Example task
task = "Find a suitable homemade pasta recipe that supports hand-twisted pasta shapes.  Provide photos of the pasta shapes."

async def generate_plan():
    # Generate plan
    plan = await planning_agent.plan(task)
    print("Generated Plan:")
    for i, step in enumerate(plan, 1):
        print(f"{i}. {step['description']}")
    return plan

# Run the async function
plan = await generate_plan()  # Changed from asyncio.run() to await

In [ ]:
# Execute plan
async def execute_plan():
    results = []
    for step in plan:
        print(f"\nExecuting: {step['description']}")  # Access the description from the step dict
        result = await execution_agent.execute(step)  # Changed to execute() and added await
        results.append(result)
        print(f"Result: {result}")
    
    # Store results in vector memory
    vector_memory.add_memory(str(results))  # Changed from add_document to add_memory
    return results

# Run the execution
results = await execute_plan()  # Changed from asyncio.run() to await